In [188]:
job_input_form_file_name = "Job Input Form.xlsm"
job_input_form_sheet_name = "Hist. Data"
non_std_data_file_name = "non_std_data.xlsx"

In [199]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from textblob import TextBlob
import pandas as pd
import numpy as np
import json
import re
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option("display.max_colwidth", None)

In [191]:
non_std_df = pd.read_excel(non_std_data_file_name)
std_df = pd.read_excel(job_input_form_file_name, sheet_name = job_input_form_sheet_name)
std_df["Job ID"] = std_df["Job ID"].apply(lambda x: x.lower())

C:\Users\huaialin\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## Help Functions

In [192]:
from nltk.corpus import stopwords
import string
def remove_digit(s):
    trans_table = str.maketrans('', '', string.digits)
    char_str = s.translate(trans_table)
    return char_str
    
        
def pre_process(row):
    """
    1. Remove all encoding strings
    2. Remove '\n'
    3. Remove underscore "_" to obtain tokenized words
    """
    stop_words = set(stopwords.words('english'))
    row = row.lower()
    row = re.sub("[a-zA-Z0-9_.-]{8,}", "", row)
    row = re.sub(r'[^\w\s]','',row)
    words = TextBlob(row).words
    words = [word for word in words if not word in stop_words]
    row = " ".join(words).replace("'", "").replace('"', "")
    row = row.replace("_", "")
    row = row.replace("\n", "")
    return row

def find_description(x):
    x = str(x).lower()
    start = x.find("problem description") + len("problem description")
    end = x.find("analysis summary")
    return x[start:end]

In [193]:
non_std_df["job id"] = non_std_df["Slide1"].apply(lambda x: str(x) if x else "")
non_std_df["job id"]  = non_std_df["job id"].apply(lambda x: 
                                                   re.search("[a-z]{2}\d{7}[a-z]{2}", str(x).lower())[0] if re.search("[a-z]{2}\d{7}[a-z]{2}", str(x).lower()) else "")

non_std_df["original problem description"] = non_std_df["Slide2"].apply(lambda x: find_description(x))
non_std_df["problem description"] = non_std_df["original problem description"].apply(lambda x: pre_process(x))

In [194]:
non_std_df2 = non_std_df[["problem description","original problem description", "job id"]].drop_duplicates()
cos_similarity_df = std_df.merge(non_std_df2, how = "inner", left_on = "Job ID", right_on = "job id")
cos_similarity_df = cos_similarity_df[cos_similarity_df["problem description"] != ""]
cos_similarity_df = cos_similarity_df.fillna("")

In [195]:
class SimilarPd:
    
    def __init__(self, train_df, parameters_for_search, top_result):
        assert len(list(set(list(train_df.columns) + parameters_for_search))) == len(train_df.columns), "Invalid parameter"
        self.train_df = train_df.drop_duplicates()
        #self.problem_description = train_df["problem description"].tolist
        self.problem_description = train_df[parameters_for_search].astype(str).apply(", ".join, axis = 1).to_list()
        self.original_problem_description = train_df["original problem description"].tolist()
        self.product_name = train_df["Product Name"].tolist()
        self.failure = train_df["Failure"].tolist()
        self.techniques = train_df["Techniques"].tolist()
        self.job_id = train_df["Job ID"].tolist()
        self.top_result = top_result
        self.vectorizer = CountVectorizer()
        self.tfidtransformer = TfidfTransformer()
        self.convert_tfidf()
        
    def convert_tfidf(self):
        tf_idf_matrix = self.vectorizer.fit_transform(self.problem_description)
        tf_idf_matrix = self.tfidtransformer.fit_transform(tf_idf_matrix)
        self.tf_idf_matrix = tf_idf_matrix
        
    def find_most_similar(self, problem_description, k, minimum_score):
        assert min_match_score <= 100 and match_score > 0, "Please choose a match score between 0 and 100"
        im = self.vectorizer.transform([problem_description])
        im = self.tfidtransformer.transform(im)
        cs = cosine_similarity(self.tf_idf_matrix, im)
        result = [np.rad2deg(np.arccos(float(x[0] if x[0] <= 1 else 1))) for x in cs]
        top = np.argsort(result)[:self.top_result]
        best_match = [self.job_id[i] for i in top if result[i] < (90.898989 + minimum_score * -89/99)]
        #feature_names = self.vectorizer.get_feature_names_out()
        #arg_score = im.toarray()[0]
        #doc1_top_words = "\n".join([str(i) + ". " + feature_names[idx] for i, idx in enumerate(arg_score.argsort()[::-1][:5])])
        result = self.train_df[self.train_df["Job ID"].isin([x for x in best_match])]
        result = result.sort_values(by = ["Job ID"], key = lambda x: x.map(lambda e: best_match.index(e)))
        return result
    
    def find_most_similar_avg_score(self, problem_description, product_name, k):
        im = self.vectorizer.transform(problem_description)
        im = self.tfidtransformer.transform(im)
        cs = cosine_similarity(self.tf_idf_matrix, im)
        result = [np.rad2deg(np.arccos(x[0])) for x in cs]
        best_match = list(zip(self.original_problem_description, result, self.product_name))
        #best_match = list(filter(lambda x: x[2] == product_name, best_match))
        best_match.sort(key = lambda x: x[1])
        best_match = best_match[:k]
        return np.nansum(result)/len(result)
        

# Assert that the parameters for search has to be in cos_similarity_df

In [267]:
def demo(spd, min_match_score):
    print(f"Search parameters are: \n{', '.join(parameters_for_search)}")
    print("==================================================================\n")
    print("Please enter the problem description:")
    x = input()
    result = spd.find_most_similar(x, 1, min_match_score)
    if len(result) > 0:
        print("\n===========================Result==================================")
        result.to_csv("search_result.csv")
        return result.drop(["job id", "Root Cause"], axis = 1)
    else:
        print("There is no matching result, please adjust the min_matching_score parameters for less trict matching condition.")

# Customisation

In [283]:
parameters_for_search = ["Product Name", "Test Phase", "Failure", "original problem description"]
min_match_score = 15 # 1-100
top_result = 5 # Adjust the maximum number of result displayed

In [284]:
spd = SimilarPd(cos_similarity_df, parameters_for_search, top_result)
result = demo(spd, min_match_score)
result

Search parameters are: 
Product Name, Test Phase, Failure, original problem description, Analyst(s)

Please enter the problem description:
nathan linarto navi21

===========================Result==================================


,Job ID,Quarter,Analyst(s),Product Name,Test Phase,Test Phase Variant,Time,Failure,Failure Variant,Failure Sub Variant,...,PFA Success,Techniques,FI Cost,PFA Cost,Technique Remarks,Date Finished,BU,Technology Node,problem description,original problem description
438,cs0025235sg,Q1,nathan linarto,navi21,rel,elfr,t6.5,bist,row,,...,,"{'Dynamic High Res PEM': ['Yes', ''], 'Dynamic Low Res PEM': ['Yes', ''], 'Layout Tracing': ['No', '']}",,,,2021-01-13 00:00:00,dGPU,tsmc7,elfr run11 t65 bist rowcol gross failing test failure,'\n 'elfr run11 t6.5 observed bist multiple row/col gross failure. failing test: mbist_hr_sms'\n 'failure
499,cs0025489sg,0,"nathan linarto, winson lua, vasanth somasundaram",navi21,rel,elfr,t44,scan,delay,,...,,"{'Dynamic High Res SDL': ['Yes', ''], 'Dynamic Low Res SDL': ['Yes', ''], 'Dynamic High Res PEM': ['Yes', ''], 'Dynamic Low Res PEM': ['Yes', ''], 'Dynamic Low Res LVP': ['Yes', ''], 'Layout Tracing': ['No', '']}",,,,00/00/00,dGPU,tsmc7,elfr run185 t44 scan delay extest5 pmax failure failure,'\n 'elfr run18-5 t44 observed scan delay extest5 pmax failure'\n 'failure
355,cs0024488sg,Q3,Winson,Navi21,REL,ERA,T48H,Scan Chain,GFX INTEST,GFX,...,,"{'Dynamic Low Res PEM':['Yes',''],'Dynamic High Res PEM ':['Yes',''],'Dynamic High Res LVP':['Yes','']}",84450.0,0.0,,09/25/20,dGPU,tsmc7,navi21 era batch gs06445 unit failing gfx intest 2 failure,' 'navi21 era minesweeper batch (gs06445)'\n 'unit is failing to gfx intest 2' 'failure
432,cs0024956sg,Q4,nathan linarto,mi100,rel,htol,t48,bist,column,,...,,"{'Dynamic High Res PEM': ['Yes', ''], 'Dynamic Low Res PEM': ['Yes', ''], 'Layout Tracing': ['No', '']}",,,,2020-10-23 00:00:00,dGPU,tsmc7,htol t48 bist 4 column failing test failure,'\n 'htol t48 observed bist 4 column failure. failing test: sms_all_posthr'\n 'failure
476,cs0025418sg,Q1,"gopinath ranganathan, vasanth somasundaram",navi21,rel,elfr,t6.5,bist,row,,...,,"{'Dynamic High Res PEM': ['Yes', ''], 'Dynamic Low Res PEM': ['Yes', ''], 'Layout Tracing': ['No', '']}",,,,2020-03-05 00:00:00,dGPU,tsmc7,failing test failure,' ' failing test: mbist_posthr_pbist_sms_pmin'\n 'failure
